In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import torch
import pandas as pd
import tqdm
from torch.utils.data import Dataset, DataLoader
import glob
import dfs_code
from torch_geometric.data import InMemoryDataset, Data

In [16]:
import sys
sys.path = ['../../src'] + sys.path
from dfs_transformer import EarlyStopping, DFSCodeSeq2SeqFC, smiles2graph

2021-08-30 16:50:42.476828: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt/lib:/opt/intel/tbb/lib:/opt/intel/clck/2019.0/lib:/opt/intel/compilers_and_libraries_2019/linux/lib:/opt/intel/compilers_and_libraries/linux/lib:/opt/intel/itac/2019.0.018/lib:/opt/intel/itac_2019/intel64/lib:/opt/intel/itac_latest/intel64/lib:/opt/intel/parallel_studio_xe_2019.0.045/clck_2019/lib:/opt/intel/parallel_studio_xe_2019.0.045/itac_2019/l

In [11]:
n_splits = 64
use_Hs = False
path = "../../results/pubchem/amd-server/noH/timeout60_4"

In [ ]:
for i in range(n_splits):
    dname = glob.glob(path+"/%d/min_dfs_codes_split*.json"%(i+1))[0]
    didx = int(dname.split("split")[-1][:-5])
    with open(path+"/%d/min_dfs_codes_split%d.json"%(i+1, didx), 'r') as f:
        codes = json.load(f)
    d_dict = {}
    for smiles, code in tqdm.tqdm(codes.items()):
        d = smiles2graph(smiles, useHs=use_Hs, addLoops=False, skipCliqueCheck=False)
        data = {}
        data['x'] = d.x.detach().cpu().numpy().tolist()
        data['z'] = d.z.detach().cpu().numpy().tolist()
        data['edge_attr'] = d.edge_attr.detach().cpu().numpy().tolist()
        data['edge_index'] = d.edge_index.detach().cpu().numpy().tolist()
        d_dict[smiles] = data
    with open(path+"/%d/data_split%d.json"%(i+1, didx), 'w') as f:
        json.dump(d_dict, f)

 95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉         | 147653/156185 [06:01<00:36, 232.37it/s]